# Yahoo Finance Fetcher tutorial

In [ ]:
from yf_fetcher import yf_fetcher
from utils import *

### Initalize fetcher

In [ ]:
yf_fetcher = yf_fetcher()

## Fetch price history

The primary feature of this tool is to fetch the historical price history of stocks to perform analysis. This requires a stock ticker, a start and end date, and an invertval at which to sample the prices. To view the valid intervals recognized by YF's API one can call `yf_fetcher.valid_intervals()`.

In [ ]:
print(yf_fetcher.valid_intervals())

Now we are ready to fetch the historical price data for a few stocks. In this example we fetch Boeing (BA), and Vanguard's Total Stock Market Index Fund Admiral Shares (VTSAX) stock prices at a weekly interval. YF's API takes the dates in seconds since January 1st, 1970 (Unix Time). For those who don't measure time in such a manner they can use the string to Unix time function `str_to_UT()`, which takes the datetime as a string and converts to Unix time (note the stock market clock is set to the US/Eastern timezone).

Then we call the `yf_fetcher.fetch_price_history()` function as follows (note that it returns meta data, and price data which we save to the variables meta and data respectively).

In [ ]:
stock = ['BA', 'VTSAX']
start = str_to_UT('2009-12-28 00:00:00', timezone='US/Eastern')
end = str_to_UT('2010-02-08 00:00:00', timezone='US/Eastern')
interval = '1wk'

meta, data = yf_fetcher.fetch_price_history(stock, start, end, interval,
                                            div=False, split=False,
                                            PrePost=False)

Since we provided a list of stock tickers, data is a list of pandas dataframes. Let's look at the Boeing stock prices by printing `data['BA']`

In [ ]:
data['BA']

## Fetching the fundamentals

If you perfer to do analysis based off fundamentals, then the Yahoo Finanace website is probably a more convient interface. However, our fetcher can grab the same data if so desired. For a list of "modules" one can grab print the `yf_fetcher.valid_modules()` function.

In [ ]:
print(yf_fetcher.valid_modules())

As an example, let's grab all the data about Apple (APPL) with the `yf_fetcher.fetch_fundamentals()` function, which takes the stock ticker and the modules you wish to fetch (here we grab all by again calling the `valid_modules()` method).

In [ ]:
fundamentals = yf_fetcher.fetch_fundamentals('AAPL', yf_fetcher.valid_modules())

Now our fundamentals variables is a dictionary with all the modules we request, as an example let's look at the calendar events.

In [ ]:
fundamentals['calendarEvents']

## Simple analysis

With the historical prices we can easily make charts that quickly compare thhe performance of two stocks. Here we compare Boeing (BA) to 
Vanguard's Short-Term Inflation-Protected Securities Index Fund ETF Shares (VITP). Since the data returned is a panadas dataframe, one can use the built in plotting utilities.

In [ ]:
stock = ['BA', 'VTIP']
start = str_to_UT('2020-01-01 00:00:00', timezone='US/Eastern')
end = str_to_UT('2020-12-01 00:00:00', timezone='US/Eastern')
interval = '1d'

meta, data = yf_fetcher.fetch_price_history(stock, start, end, interval,
                                    div=False, split=False, PrePost=True)

ax = (data['VTIP'].iloc[:,2]/data['VTIP'].iloc[0,2]).plot(label='VITP')
(data['BA'].iloc[:,2]/data['BA'].iloc[0,2]).plot(ax=ax, label='BA')
ax.legend()
ax.set_title('Growth since 01/01/2020')
ax.set_ylabel('Growth')

fig = ax.get_figure()
fig.autofmt_xdate()
fig.tight_layout()

We can look at VTIP growth of the close and the adjust close, by normalizing by the starting prices.

In [ ]:
data['VTIP'].iloc[:,1:3]/data['VTIP'].iloc[0,1:3]

We can also look at the true value of a stock compared to its nomial price by plotting the adjusted close and close prices of a stock, here we use VTSAX.

In [ ]:
stock = 'VTSAX'
start = str_to_UT('1960-01-01 04:00:00', timezone='US/Eastern')
end = str_to_UT('2020-01-01 20:00:00', timezone='US/Eastern')

xmeta, data = yf_fetcher.fetch_price_history(stock, start, end, '3mo',
                    div=True, split=False, PrePost=True)


ax = data.iloc[:,1:3].plot()
ax.legend()
ax.set_yscale('log')
ax.set_ylabel('{:s} ({:s})'.format(xmeta['symbol'], xmeta['currency']))

fig = ax.get_figure()
fig.autofmt_xdate()
fig.tight_layout()